# PROBLEM A3

Build a classifier for the Human or Horse Dataset with Transfer Learning.
The test will expect it to classify binary classes.
Note that all the layers in the pre-trained model are non-trainable.
Do not use lambda layers in your model.

The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
Inception_v3, pre-trained model used in this problem is developed by Google.

Desired accuracy and validation_accuracy > 97%.

In [1]:
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 80.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [2]:
!pip install pandas==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [3]:
!pip install Pillow==10.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [4]:
!pip install scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3


In [5]:
!pip install tensorflow==2.13.0

In [6]:
!pip install tensorflow-datasets==4.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.3
    Uninstalling tensorflow-datasets-4.9.3:
      Successfully uninstalled tensorflow-datasets-4.9.3


## Import

In [1]:
import numpy as np
import pandas as pd
import PIL as pil
import scipy as sc
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
pretrained_model = InceptionV3(include_top=False,
                               weights='imagenet',
                               input_shape=(150,150,3))

87910968/87910968 [==============================] - 0s 0us/step


In [24]:
train, test = tfds.load('horses_or_humans', split=['train','test'], as_supervised = True)

In [25]:
train_x = []
train_y = []
test_x = []
test_y = []

In [26]:
for images, label in train:
  images = tf.image.resize(images,[150,150])
  train_x.append(images.numpy())
  train_y.append(label.numpy())
for images, label in test:
  images = tf.image.resize(images,[150,150])
  test_x.append(images.numpy())
  test_y.append(label.numpy())

In [27]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [28]:
train_x[0]

array([[[ 47.25,  47.5 ,  40.25],
        [ 41.  ,  41.75,  37.5 ],
        [ 30.  ,  32.75,  26.75],
        ...,
        [ 27.25,  28.5 ,  27.25],
        [ 21.5 ,  23.  ,  22.  ],
        [ 18.  ,  20.  ,  18.25]],

       [[ 40.  ,  37.  ,  34.25],
        [ 47.  ,  45.5 ,  41.25],
        [ 24.75,  27.5 ,  22.75],
        ...,
        [ 18.75,  21.75,  22.25],
        [ 17.  ,  19.25,  18.25],
        [ 13.75,  15.  ,  13.25]],

       [[ 46.  ,  44.25,  38.5 ],
        [ 53.75,  55.  ,  43.5 ],
        [ 24.5 ,  27.5 ,  23.  ],
        ...,
        [ 15.75,  17.5 ,  17.25],
        [ 14.  ,  16.  ,  15.5 ],
        [ 12.75,  14.25,  13.  ]],

       ...,

       [[123.  , 110.25,  97.25],
        [128.25, 110.25,  93.75],
        [103.75,  91.  ,  76.25],
        ...,
        [109.5 , 100.5 ,  91.  ],
        [149.  , 139.  , 129.5 ],
        [153.5 , 135.75, 120.25]],

       [[154.25, 140.5 , 123.75],
        [143.25, 128.  , 109.5 ],
        [125.  , 111.  ,  96.75],
        .

In [29]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.,
                                                            horizontal_flip=True,
                                                            zoom_range=0.2,
                                                            fill_mode='nearest',
                                                            height_shift_range=0.2,
                                                            width_shift_range=0.2,
                                                            rotation_range=30)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.,
                                                           fill_mode='nearest')

In [30]:
train_datagen = train_gen.flow(train_x, train_y)
test_datagen = test_gen.flow(test_x, test_y)

Layer Selection

In [31]:
# Freeze all of the layers
for layer in pretrained_model.layers:
  layer.trainable = False

In [32]:
# Use the mixed7 layer for the transfer learning, in InceptionV3 the mixed7 layer is the most optimal layer to use transfer learning
last_layer = pretrained_model.get_layer('mixed7')

In [33]:
# get the output of the layer to be used in our model
last_output = last_layer.output

Create the model

In [34]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x) # YOUR CODE HERE, BUT END WITH A Neuron Dense, activated by sigmoid
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [35]:
model = tf.keras.Model(pretrained_model.input,
                       x)

In [36]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [37]:
model.fit(train_datagen, validation_data=test_datagen,epochs=8)

Epoch 1/8
33/33 [==============================] - 15s 230ms/step - loss: 0.3488 - accuracy: 0.8656 - val_loss: 0.0693 - val_accuracy: 0.9961
Epoch 2/8
33/33 [==============================] - 8s 236ms/step - loss: 0.1003 - accuracy: 0.9796 - val_loss: 0.0327 - val_accuracy: 0.9883
Epoch 3/8
33/33 [==============================] - 6s 189ms/step - loss: 0.0565 - accuracy: 0.9883 - val_loss: 0.0113 - val_accuracy: 0.9961
Epoch 4/8
33/33 [==============================] - 9s 286ms/step - loss: 0.0416 - accuracy: 0.9873 - val_loss: 0.0079 - val_accuracy: 0.9961
Epoch 5/8
33/33 [==============================] - 6s 171ms/step - loss: 0.0212 - accuracy: 0.9951 - val_loss: 0.0140 - val_accuracy: 0.9922
Epoch 6/8
33/33 [==============================] - 7s 224ms/step - loss: 0.0149 - accuracy: 0.9961 - val_loss: 0.0095 - val_accuracy: 0.9922
Epoch 7/8
33/33 [==============================] - 6s 180ms/step - loss: 0.0142 - accuracy: 0.9951 - val_loss: 0.0090 - val_accuracy: 0.9961
Epoch 8/8
33